In [ ]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'Home.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [ ]:
from basemodels.models import Tournament, Season, Team,Match
from basemodels.models import FikstureModelData
from datetime import datetime, timedelta

class SofascoreAPI:
    def __init__(self):
        pass

    def fetch_data(self):
        data = FikstureModelData.objects.all().values()
        data_list = list(data)
        df = pd.DataFrame(data_list)
        df_exploded = df.explode('data')
        expanded_data = pd.json_normalize(df_exploded['data'])
        expanded_data['tarih'] = list(df_exploded['tarih'])
        veri = expanded_data.copy()
        veri.columns = [item.replace(".", "_") for item in veri.columns]
        return veri

    def tournament_data(self):
        verim = self.fetch_data()
        if verim is None:
            raise ValueError("Veri işlenmedi. Önce 'process_data()' metodunu çalıştırın.")
        
        # Aynı uniqueTournament_id'ye sahip olanları tekrar kontrol et
        verim = verim.drop_duplicates("tournament_uniqueTournament_id")
        
        cols = [
            "tournament_uniqueTournament_id", "tournament_name", "tournament_slug", "tournament_category_name",
            "tournament_category_slug", "tournament_uniqueTournament_category_id", 
            "tournament_uniqueTournament_hasEventPlayerStatistics", "tournament_id", "tournament_isGroup", 
            "tournament_uniqueTournament_hasPerformanceGraphFeature"
        ]
        
        for col in cols:
            if col not in verim.columns:
                verim.loc[:, col] = 0

        verim = verim[cols]

        for _, row in verim.iterrows():
            tournament_id = row['tournament_uniqueTournament_id']
            
            if pd.isna(tournament_id):
                print(f"Geçersiz tournament_uniqueTournament_id: {tournament_id}, atlanıyor.")
                continue
        
            if not Tournament.objects.filter(tournament_id=tournament_id).exists():
                Tournament.objects.create(
                    tournament_id=row['tournament_uniqueTournament_id'],
                    tournament_name=row['tournament_name'],
                    tournament_slug=row['tournament_slug'],
                    tournament_category_name=row['tournament_category_name'],
                    tournament_category_slug=row['tournament_category_slug'],
                    tournament_uniqueTournament_category_id=row['tournament_uniqueTournament_category_id'],
                    tournament_model_id=row['tournament_id'],
                    tournament_isGroup=row['tournament_isGroup'] if pd.notna(row['tournament_isGroup']) else False,
                    tournament_uniqueTournament_hasPerformanceGraphFeature=row['tournament_uniqueTournament_hasPerformanceGraphFeature'] if pd.notna(row['tournament_uniqueTournament_hasPerformanceGraphFeature']) else False
                )
                print(f"Tournament ID {tournament_id} başarıyla eklendi.")
            else:
                print(f"Tournament ID {tournament_id} zaten mevcut, eklenmedi.")

    def season_data(self):
        verim = self.fetch_data()
        if verim is None:
            raise ValueError("Veri işlenmedi. Önce 'process_data()' metodunu çalıştırın.")
        
        verim = verim.drop_duplicates("season_id")
        
        season_cols = ["season_id", "season_name", "season_year","tournament_uniqueTournament_id"]

        for col in season_cols:
            if col not in verim.columns:
                verim.loc[:, col] = 0

        verim = verim[season_cols]

        for _, row in verim.iterrows():
            season_id = row['season_id']
            tournament_id = row.get('tournament_uniqueTournament_id')
    
            if pd.isna(season_id) or pd.isna(tournament_id):
                print(f"Geçersiz season_id veya tournament_id: Season ID={season_id}, Tournament ID={tournament_id}. Atlanıyor.")
                continue
    
            if not Season.objects.filter(season_id=season_id).exists():
                tournament_instance = Tournament.objects.filter(tournament_id=tournament_id).first()
                
                if tournament_instance:
                    Season.objects.create(
                        season_id=row['season_id'],
                        season_name=row['season_name'],
                        season_year=row['season_year'],
                        tournament=tournament_instance  # Doğru sütun adı burada kullanılıyor
                    )
                    print(f"Season ID {season_id} başarıyla eklendi.")
                else:
                    print(f"Tournament ID {tournament_id} bulunamadı, Season ID {season_id} eklenemedi.")
            else:
                print(f"Season ID {season_id} zaten mevcut, eklenmedi.")
    
   
    def team_data(self):

        data = self.fetch_data()
        homecols = [
                    "homeTeam_id", "homeTeam_name", "homeTeam_slug", "homeTeam_shortName",
                    "homeTeam_nameCode", "homeTeam_national", "tournament_uniqueTournament_id", "season_id"
                ]
        
        awaycols = [
                    "awayTeam_id", "awayTeam_name", "awayTeam_slug", "awayTeam_shortName",
                    "awayTeam_nameCode", "awayTeam_national", "tournament_uniqueTournament_id", "season_id"
                ]
        
        # Home takımı için veri seçimi ve kolon isimlerinin ayarlanması
        homedata = data[homecols]
        homedata.columns = ["team_id", "team_name", "team_slug", "team_shortName",
                                       "team_nameCode", "team_national", "tournament_id", "season_id"]
        
        # Away takımı için veri seçimi ve kolon isimlerinin ayarlanması
        awaydata = data[awaycols]
        awaydata.columns  = ["team_id", "team_name", "team_slug", "team_shortName",
                                       "team_nameCode", "team_national", "tournament_id", "season_id"]
        
        # Home ve Away takımları birleştirme
        teamdata = pd.concat([homedata, awaydata])
        teamdata = teamdata.drop_duplicates(subset=["team_id", "tournament_id", "season_id"])
       
        for _, row in teamdata.iterrows():
            team_id = row['team_id']
            tournament_id = row['tournament_id']
            season_id = row['season_id']
            
            # Geçersiz veya eksik id'leri atla
            if pd.isna(team_id) or pd.isna(tournament_id) or pd.isna(season_id):
                print(f"Geçersiz değer: Team ID={team_id}, Tournament ID={tournament_id}, Season ID={season_id}. Atlanıyor.")
                continue

            # İlgili tournament ve season nesnelerini getir
            tournament_instance = Tournament.objects.filter(tournament_id=tournament_id).first()
            season_instance = Season.objects.filter(season_id=season_id).first()

            if tournament_instance and season_instance:
                # Takımın veritabanında var olup olmadığını kontrol et, yoksa oluştur
                if not Team.objects.filter(team_id=team_id, tournament=tournament_instance, season=season_instance).exists():
                    Team.objects.create(
                        team_id=team_id,
                        team_name=row['team_name'],
                        team_slug=row['team_slug'],
                        team_shortName=row['team_shortName'],
                        team_nameCode=row['team_nameCode'],
                        team_national=row['team_national'],
                        tournament=tournament_instance,  # ForeignKey olan alanlar
                        season=season_instance           # ForeignKey olan alanlar
                    )
                    print(f"Team ID {team_id} başarıyla eklendi.")
                else:
                    print(f"Team ID {team_id} zaten mevcut, eklenmedi.")
            else:
                print(f"Tournament ID {tournament_id} veya Season ID {season_id} bulunamadı.")

    def match_data(self):
        # Fetch data from the API or existing model
        verim = self.fetch_data()
        if verim is None:
            raise ValueError("Veri işlenmedi. Lütfen önce 'process_data()' metodunu çalıştırın.")
        
     
     
        new_cols = ['id', 'startTimestamp', 'slug', 'tournament_uniqueTournament_id', 'season_id',
                    'roundInfo_round', 'status_type', 'homeTeam_id', 'awayTeam_id',
                    'homeScore_current', 'homeScore_period1', 'homeScore_period2', 'homeScore_normaltime',
                    'awayScore_current', 'awayScore_period1', 'awayScore_period2', 'awayScore_normaltime']
        
        df1 = verim[new_cols] 
        df1['date'] = pd.to_datetime(df1['startTimestamp'], unit='s') + timedelta(hours=3)
        df2 = df1.drop("startTimestamp", axis=1)
        df3 = df2.fillna(0)
        for _, row in df3.iterrows():
            match_id = row['id']
            tournament_id = row['tournament_uniqueTournament_id']
            season_id = row['season_id']
            home_team_id = row['homeTeam_id']
            away_team_id = row['awayTeam_id']
            
            # Geçersiz veya eksik id'leri atla
            if pd.isna(match_id) or pd.isna(tournament_id) or pd.isna(season_id) or pd.isna(home_team_id) or pd.isna(away_team_id):
                print(f"Geçersiz değer: Match ID={match_id}, Tournament ID={tournament_id}, Season ID={season_id}, Home Team ID={home_team_id}, Away Team ID={away_team_id}. Atlanıyor.")
                continue

            # İlgili foreign key nesnelerini getir
            tournament_instance = Tournament.objects.filter(tournament_id=tournament_id).first()
            season_instance = Season.objects.filter(season_id=season_id).first()
            home_team_instance = Team.objects.filter(team_id=home_team_id).first()
            away_team_instance = Team.objects.filter(team_id=away_team_id).first()

            if tournament_instance and season_instance and home_team_instance and away_team_instance:
                # Match'in veritabanında olup olmadığını kontrol et, yoksa oluştur
                if not Match.objects.filter(id=match_id).exists():
                    Match.objects.create(
                        id=match_id,
                        startTimestamp=row['date'],  # DateTimeField olarak datetime verisi
                        slug=row['slug'],
                        tournament=tournament_instance,
                        season=season_instance,
                        homeTeam=home_team_instance,
                        awayTeam=away_team_instance,
                        roundInfo_round=row['roundInfo_round'],
                        homeScore_current=row['homeScore_current'],
                        homeScore_period1=row['homeScore_period1'],
                        homeScore_period2=row['homeScore_period2'],
                        homeScore_normaltime=row['homeScore_normaltime'],
                        awayScore_current=row['awayScore_current'],
                        awayScore_period1=row['awayScore_period1'],
                        awayScore_period2=row['awayScore_period2'],
                        awayScore_normaltime=row['awayScore_normaltime'],
                        status_type = row['status_type']
                    )
                    print(f"Match ID {match_id} başarıyla eklendi.")
                else:
                    print(f"Match ID {match_id} zaten mevcut, eklenmedi.")
            else:
                print(f"Tournament ID {tournament_id}, Season ID {season_id}, Home Team ID {home_team_id}, Away Team ID {away_team_id} bulunamadı.")





# SofascoreAPI sınıfını kullanarak team_data'yı veritabanına kaydetmek için:
api = SofascoreAPI()
teamdata = api.fetch_data()  # Veriyi getirin (API'den ya da dosyadan)
#api.tournament_data()
#api.season_data()
#api.team_data()
api.match_data()
print("İşlemler bitti")
